## SDKを用いた計装

必要なライブラリをダウンロードします。

In [ ]:
%pip install -r ../requirements.txt

実行に必要な環境変数を読み込みます。

In [2]:
import os, warnings, time

# おまじない
warnings.filterwarnings("ignore")

endpoint = "http://localhost:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

LangfuseのSDKクライアントを初期化します。

In [3]:
from langfuse import Langfuse

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

### Decorator

`@obserbe()`デコレーターを用いてTraceと各種Observation(Span/Generation)の確認をします。

In [4]:
from langfuse.decorators import observe, langfuse_context

@observe()
def meet_the_bigbaby():
    print("Meet the BigBaBy!")

@observe(name="Search the BigBaBy's dictionary")
def search_the_bigbaby_dict():
    time.sleep(2)

@observe(name="Ask the BigBaBy", as_type="generation")
def ask_the_bigbaby(query: str):
    if os.getenv("COHERE_API_KEY") == None:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        model = "gpt-4o-mini"
        llm = ChatOpenAI(api_key=openai_api_key, model=model)
    else:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        model = "command-r-plus"
        llm = ChatCohere(cohere_api_key=cohere_api_key, model=model)
    input = [
        {"role": "user", "content": query}
    ]
    result = llm.invoke(input=input)
    return result.content

@observe(name="Ask the BigBaby")
def main():
    meet_the_bigbaby()
    search_the_bigbaby_dict()
    return ask_the_bigbaby(query="肉料理のおすすめ教えて")

langfuse_context.configure(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)
response = main()
print(f"{response=}")

Meet the BigBaBy!
response='喜んでいただけるか分かりませんが、いくつかおすすめの肉料理を紹介します。\n\n- ローストビーフ： 牛肉の塊をオーブンで低温調理し、薄くスライスして食べる料理です。ジューシーで柔らかく、赤身の美味しさを味わえます。\n- 豚の角煮： 豚バラ肉を醤油や砂糖などで長時間煮込んだ料理です。トロトロに柔らかくなったお肉と、染み込んだタレの味がご飯に良く合います。\n- チキンティッカ： インド料理のグリルチキンです。ヨーグルトや香辛料に漬け込んだ鶏肉を串に刺して焼きます。香り豊かでジューシーな味わいが特徴です。\n- ラムチョップ： 仔羊のリブ（あばら骨）の部分をグリルしたものです。脂がのったジューシーなお肉と、独特の香りがクセになります。\n- 牛タン焼き： 牛タンをスライスして焼き、塩コショウなどで味付けしたシンプルな料理です。柔らかくて旨味たっぷりのお肉を楽しめます。\n\n以上、お口に合うものが見つかるといいですね。'


### Low-Level SDK

#### 属性情報を付与しない実行例

Low-Level SDKを用いてTraceと各種Observation(Event/Span/Generation)の確認をします。

In [5]:
# Traceを作成する
trace = langfuse.trace(
    name="Ask the BigBaBy"
)

# Observation - EventをTraceに追加する
event = trace.event(
    name="Meet the BigBaBy"
)

# Observation - SpanをTraceに追加する
span = trace.span(
    name="Search the BigBaBy's dictionary"
)
# 2秒間スリープ（同期コールする検索システムを想定）
time.sleep(2)
span.end()

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.chat_models import ChatOpenAI
    openai_api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-mini"
    llm = ChatOpenAI(api_key=openai_api_key, model=model)
else:
    from langchain_cohere.chat_models import ChatCohere
    cohere_api_key = os.getenv("COHERE_API_KEY")
    model = "command-r-plus"
    llm = ChatCohere(cohere_api_key=cohere_api_key, model=model)

input = [
    {"role": "user", "content": "おすすめの肉料理おしえて"}
]

generation = trace.generation(
    name="Ask the BigBaBy",
    model=model,
    input=input
)

result = llm.invoke(input=input)

generation.end(
    output=result.content,
)

#### 属性情報を付与した実行例

属性情報などを追加した版です。

In [6]:
# Traceを作成する
trace = langfuse.trace(
    name="Ask the BigBaBy"
)

trace_id = trace.id

# Observation - EventをTraceに追加する
event = trace.event(
    name="Meet the BigBaBy"
)

# Observation - SpanをTraceに追加する
span = trace.span(
    name="Search the BigBaBy's dictionary"
)
# 2秒間スリープ（同期コールする検索システムを想定）
time.sleep(2)
# ダミーの検索結果
search_result = {
    "recipes": [{
        "roast-beaf": "ローストビーフ： 牛肉の塊をオーブンで焼いた料理で..."
    }]
}
span.end()
span.update(
    input="おすすめの肉料理おしえて",
    output=search_result
)

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.chat_models import ChatOpenAI
    openai_api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-mini"
    llm = ChatOpenAI(api_key=openai_api_key, model=model)
else:
    from langchain_cohere.chat_models import ChatCohere
    cohere_api_key = os.getenv("COHERE_API_KEY")
    model = "command-r-plus"
    llm = ChatCohere(cohere_api_key=cohere_api_key, model=model)

input = [
    {"role": "user", "content": "おすすめの肉料理おしえて"}
]

generation = trace.generation(
    name="Ask the BigBaBy",
    model=model,
    input=input
)

result = llm.invoke(input=input)

generation.end(
    output=result.content,
)

trace.update(
    input="おすすめの肉料理おしえて",
    output=result.content
)